In [27]:
import numpy as np
from numpy import cos, sin
import matplotlib.pyplot as plt
from libs.XY_model import XYSystem
import cv2
from tqdm import tqdm

In [28]:
def get_curl(x, neighbors):    
    N = x.shape[0]
    curl = np.zeros([N, N])
    for i in range(N):
        for j in range(N):
            for a in range(1, neighbors):
                curl[i, j] += (  ( (2 * sin(x[i, j]) - sin(x[i, (j+a)%N] - sin(x[i, (j-a)%N])))) -
                                 ( (2 * cos(x[i, j])) - cos(x[(i+a)%N, j]) - cos(x[(i-a)%N, j]))   )/a
                curl[i, j] += (  ( (2 * sin(x[i, j]) - sin(x[(i+a)%N, (j+a)%N] - sin(x[(i-a)%N, (j-a)%N])))) -
                                 ( (2 * cos(x[i, j])) - cos(x[(i+a)%N, (j+a)%N]) - cos(x[(i-a)%N, (j-a)%N]))   )/(a * (2**0.5))
                curl[i, j] += (  ( (2 * sin(x[i, j]) - sin(x[(i-a)%N, (j+a)%N] - sin(x[(i+a)%N, (j-a)%N])))) -
                                 ( (2 * cos(x[i, j])) - cos(x[(i-a)%N, (j+a)%N]) - cos(x[(i+a)%N, (j-a)%N]))   )/(a * (2**0.5))
    curl /= neighbors
    return curl

In [29]:
def list2matrix(S):
        N=int(np.size(S))
        L = int(np.sqrt(N))
        S=np.reshape(S,(L,L))
        return S

In [30]:
def plot_heatmap(spin_config, curl, temp, name = 'result', heat = True, just_heat = False):
    N = spin_config.shape[0]
    X, Y = np.meshgrid(np.arange(0,N),np.arange(0, N))
    U = np.cos(spin_config)
    V = np.sin(spin_config)
    plt.figure(figsize=(4,4),dpi=800, facecolor='white')
    if(heat):
        plt.imshow(curl, extent=(0, N, 0, N))
        plt.colorbar()
    if(just_heat):
        plt.imshow(curl,extent=(0, N, 0, N))
        plt.colorbar()
        plt.savefig(f'./images/{name}.png', dpi = 800)
        plt.close()
        return
    Q = plt.quiver(X, Y, U, V, units='width')
    qk = plt.quiverkey(Q, 0.1, 0.1, 1, r'$spin$', labelpos='E',
                coordinates='figure')
    plt.title('T=%.2f'%temp+', #spins='+str(N)+'x'+str(N))
    plt.savefig(f'./images/{name}.png', dpi = 800)
    plt.close()

In [31]:
temp = [0.01, 0.05, 0.08, 0.1, 0.2, 0.6]
N = [32]

In [32]:
for n in tqdm(range(len(N))):    
    for i in range(len(temp)):
        xy1 = XYSystem(temperature = temp[i], width = N[n])
        xy1.equilibrate(show=False)
        x = list2matrix(xy1.spin_config)
        cu = get_curl(x, (int)(x.shape[0]/5))
        plot_heatmap(x, cu, temp[i], f'mix_{temp[i]}_{N[n]}')
        plot_heatmap(x, cu, temp[i], f'vec_{temp[i]}_{N[n]}', False)
        plot_heatmap(x, cu, temp[i], f'curl_{temp[i]}_{N[n]}', False, True)

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 